In [1]:
import pandas as pd
import folium
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import openpyxl

In [2]:
clinics = pd.DataFrame()

url = "http://apis.data.go.kr/B552657/HsptlAsembySearchService/getHsptlMdcncListInfoInqire"
ServiceKey = "?ServiceKey=(서비스키)"
PageNo = "&pageNo="
numOfRows = "&numOfRows="

for a in range(1,705):
    b = 100
    url_query = url + ServiceKey + PageNo + str(a) + numOfRows + str(b)
    page = urlopen(url_query)
    soup = BeautifulSoup(page, "html.parser")
    block = soup.find('items')
    for i in range(0,b):
        eachitem = block.find_all('item')[i]
        name = (eachitem.find('dutyname').string if eachitem.find('dutyname') else '')
        lat = (eachitem.find('wgs84lat').string if eachitem.find('wgs84lat') else '')
        lon = (eachitem.find('wgs84lon').string if eachitem.find('wgs84lon') else '')
        new = pd.DataFrame([{'name':name,'lat':lat,'lon':lon}])
        clinics = pd.concat([clinics,new])
       
clinics.head()

,lat,lon,name
0,35.88758663796097,128.65086512206707,대구광역시동구보건소
0,35.8007916146,128.5326926621,수목원치과의원(7/6부재중)(정보확인요망)
0,35.86993902090882,128.69192818752236,(VOM)봄안과의원
0,35.14964277738037,126.88689410104307,(공동체의료소비자생활협동조합)비타민의원
0,35.09731743229667,129.01328806613628,(복지)예사랑병원


In [3]:
clinics.shape

(70400, 3)

In [4]:
clinics.dtypes

lat     object
lon     object
name    object
dtype: object

In [5]:
clinics['lat'] = pd.to_numeric(clinics['lat'], errors='coerce').fillna(0)
clinics['lon'] = pd.to_numeric(clinics['lon'], errors='coerce').fillna(0)

In [7]:
clinics.dtypes

lat     float64
lon     float64
name     object
dtype: object

In [8]:
clinics2 = clinics.reset_index(drop=True)

In [9]:
clinics2 = clinics2.drop(clinics2[clinics2.lon < 20].index)
clinics2.shape

(70373, 3)

In [10]:
clinics2.to_excel('./allclinics2.xlsx')

In [11]:
clinics2 = clinics[~clinics['name'].str.contains('한방')]
clinics2 = clinics[~clinics['name'].str.contains('한의원')]
clinics2.shape

(56268, 3)

In [12]:
skin = clinics2[clinics2['name'].str.contains('피부')].reset_index(drop=True)
skin.shape

(1294, 3)

In [13]:
locations = skin[['lat', 'lon']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[700]

[37.48858314909423, 127.05140422712152]

In [14]:
skin.head()

,lat,lon,name
0,35.888743,128.591848,2080연합피부과의원
1,36.812377,127.108192,CNP차앤박피부과의원
2,35.148700,126.848925,CNP차앤박피부과의원
3,37.446192,126.700919,CNP차앤박피부과의원
4,36.425952,127.387207,DS피부과의원


In [15]:
map = folium.Map(location=[36.536236, 127.5], zoom_start=8)

folium.TileLayer('stamenterrain').add_to(map)
folium.TileLayer('stamentoner').add_to(map)
folium.TileLayer('cartodbpositron').add_to(map)
folium.LayerControl().add_to(map)
for i in range(0, len(locationlist)):
    folium.CircleMarker(locationlist[i], radius=6,
                        color='purple', fill=True, fill_color='white',
                        tooltip=skin['name'][i]).add_to(map)
    
##for i in range(0, len(locationlist)):
##    folium.Marker(locationlist[i], tooltip=skin['name'][i]).add_to(map)
    
map.save('D:\Dropbox\code\map_skin.html')